# Testing functions in telo_tools for handling individual telomere length data

In [23]:
# enables access to directories/files
import os

# for handling data
import numpy as np
from numpy import array
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

# graphing
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns

# for loading telo data column containing individual telomere length values
from ast import literal_eval

# statistics
from scipy import stats

# accessing telo_tools module, which is in the parent directory, for testing
import sys 
sys.path.append('..')
import telo_tools

In [8]:
# fxn for reading files
# test_df['telo data'] = test_df['telo data'].apply(lambda row: np.array(literal_eval(row)))

## Test compiling individual telomere length data and generating data into a dataframe

In [28]:
# compiling test data from excel files into a dict
test_dict = telo_tools.generate_dictionary_for_telomere_length_data('.')

xyz9999_FD45.xlsx telomere data acquisition in progress..
xyz9999_L-180.xlsx telomere data acquisition in progress..
xyz9999_L-60.xlsx telomere data acquisition in progress..
Done collecting all telomere length excel files


In [12]:
# generating a pandas dataframe from the dict
test_df = telo_tools.make_dataframe_from_telomere_data_dict(test_dict)
test_df.head(4)

,sample id,timepoint,telo data,telo means,Q1,Q2-3,Q4
0,9999,FD45,4537 6589.0 1005 11666.0 2797 11540....,9273.515870,telos quartile 1 <0.25,telos quartile 2-3 >0.25 & <0.75,telos quartile 4 >0.75
1,9999,L-180,0 5802.0 1 18223.0 2 10506....,10317.742572,telos quartile 1 <0.25,telos quartile 2-3 >0.25 & <0.75,telos quartile 4 >0.75
2,9999,L-60,0 14177.0 1 11729.0 2 9170....,10801.377174,telos quartile 1 <0.25,telos quartile 2-3 >0.25 & <0.75,telos quartile 4 >0.75


In [27]:
import importlib
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Counting short/medium/long individual telomeres

In [15]:
# declaring a list of 
timepoint_list = ['L-180', 'L-60', 'FD45', 'FD90', 'FD140', 'FD260', 'R+5', 'R+7', 'R+60', 'R+105', 'R+180', 'R+270']

telo_tools.calculate_apply_teloQuartiles_dataframe(test_df, timepoint_list)

,sample id,timepoint,telo data,telo means,Q1,Q2-3,Q4
0,9999,L-180,0 5802.0 1 18223.0 2 10506....,10317.742572,[1380],[2760],[1380]
1,9999,L-60,0 14177.0 1 11729.0 2 9170....,10801.377174,[1328],[2460],[1732]
2,9999,FD45,4537 6589.0 1005 11666.0 2797 11540....,9273.515870,[1645],[2289],[666]


## Exploding individual telomere length measurements into the dataframe as rows

In [19]:
def explode_individual_telos(df):
    exploded_telos = (df['telo data'].apply(pd.Series)
        .merge(df, right_index = True, left_index = True)                  
        .drop('telo data', axis=1)
        .melt(id_vars = [col for col in df.columns if col != 'telo data'], value_name = "individual telos") 
        .drop("variable", axis = 1)
        .dropna())
    
    exploded_telos['individual telos'] = exploded_telos['individual telos'].astype('int64')
    
    return exploded_telos

In [21]:
# explode_individual_telos(test_df)